In [1]:
import numpy as np
import numba as nb

/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numba/errors.py:102: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
@nb.jit(nopython=True)
def locate(x, grid, btm = 0, up = None):
    if up is None:
        up = len(grid)-1
    
    """
    from Numerical Repipes 2nd ed.
    input
    x: a value to be evaluated
    grid: a monotonically ordered grid
    
    return grid number 0,1,....,len(grid) -2
    """
    
    N = len(grid)
    
    
    
    if grid[N-1] > grid[0]:
        if x <= grid[0]:
            return 0
        elif x >= grid[N-1]:
            return N-2
    elif grid[N-1] < grid[0]:
        if x <= grid[N-2]:
            return N-2
        elif x >= grid[0]:
            return 0
        
    else:
        print('error: grid[N-1] == grid[0]')
    
    #Golden search
    mid = 0
    while up - btm > 1: #if not done
        mid = int((up+btm)/2) #math.floor? 
#         if (grid[N-1] > grid[0]) == (x > grid[mid]):
        if (grid[N-1] > grid[0]) == (x > grid[mid]):
            
            btm = mid
        else:
            up = mid

    if up - btm < 1: 
        print('locate, error: up - btm < 1, up = ', up, ', btm = ', btm, '.')

    return btm   


@nb.jit(nopython=True)
def hunt(x, grid, init_btm):
    N = len(grid)
    ascnd = (grid[N-1] > grid[0])
    btm = init_btm
    up = 0#None does not work for numba
    
    if init_btm < 0 or init_btm > N-2:
        return locate(x,grid)
    else:
        inc = 1 #increment
        if (x > grid[btm]) is ascnd:
            while True:
                up = btm + inc
                #print('up: ',up)
                if up > N-2:
                    up = N-1
                    #print('up: ',up)
                    break
                elif (x > grid[up]) is ascnd:
                    btm = up
                    inc = inc + inc
                else:
                    break
        else:
            up = btm
            #print('up: ',up)
            while True:
                btm = up - inc
                #print('btm: ',btm)
                if btm < 0:
                    btm = 0
                    #print('btm: ',btm)
                    break
                elif (x < grid[btm]) is ascnd:
                    up = btm
                    inc = inc + inc
                    #print('up: ',up)
                else:
                    break
        #print('btm: ',btm)
        #print('up: ',up)        
        return locate(x, grid, btm, up)
    

    
#@nb.jit#(nopython=True)#hasattr is not compatible with numba

#this should accept an array-like with len == 1
@nb.jit(nopython=True)
def locate_on_grids(xvals, grid, init_btm = 0):
    M = len(xvals)
    ans = np.zeros(M, dtype = np.int64)
    ans[0]= hunt(xvals[0], grid, init_btm)

    for ix in range(1,M):
        ans[ix] = hunt(xvals[ix], grid, ans[ix-1])
        #thiw was a typo. can't set btm = ans[ix-1]
        #ans[ix] = locate(xvals[ix], grid, ans[ix-1])  #i donno why but it is faster

    return ans

@nb.generated_jit(nopython=True)
def locate_grid(xvals, grid, init_btm = 0, return_nparray = False):
    

    if isinstance(xvals, nb.types.Float) or isinstance(xvals, nb.types.Integer) : #if xvals is scalar
        #here, locate is converted into np.array
        #if you need just an interger, use locate instead.
        return lambda xvals, grid, init_btm, return_nparray: np.array(locate(xvals, grid))
        
        #if return_nparray is True:
        #    return lambda xvals, grid, init_btm, return_nparray: np.array(locate(xvals, grid))
        #else:
        #    return lambda xvals, grid, init_btm, return_nparray: locate(xvals, grid)
    
    else: #arraylike #maybe I should check this is arraylike
        return lambda xvals, grid, init_btm, return_nparray: locate_on_grids(xvals, grid, init_btm)


In [5]:
if __name__ == '__main__':
    
    import time
    
    size = 100000
    
    bignodes = np.linspace(-10, 100, size)
    xvals = np.linspace(-1, 50, size)
    t1 = time.time()
    hunt(6.6, bignodes, 3)

    t2 = time.time()

    print(' {} seconds'.format(t2 - t1))


    #comapred the speed
    t1 = time.time()
    M = len(xvals)
    ans1 = np.zeros(M, dtype = np.int64)

    for ix, x in enumerate(xvals):
        ans1[ix] = locate(x, bignodes)

    t2 = time.time()

    print(' {} seconds'.format(t2 - t1))
    
    
    
    t1 = time.time()
    M = len(xvals)
    ans2 = np.zeros(M, dtype = np.int64)

    ans2[0] = hunt(x, bignodes, 0)
    for ix in range(M):
        x = xvals[ix]
        ans2[ix] = hunt(x, bignodes, ans2[ix-1]+1)

    t2 = time.time()

    print(' {} seconds'.format(t2 - t1))
    
    
    #comapred the speed
    t1 = time.time()

    ans3 = locate_grid(xvals, bignodes)

    t2 = time.time()

    print(' {} seconds'.format(t2 - t1))


 7.152557373046875e-06 seconds
 0.10769414901733398 seconds
 0.11383700370788574 seconds
 0.008481979370117188 seconds


In [12]:
 8.106231689453125e-06 seconds
 0.09943079948425293 seconds
 0.11290311813354492 seconds
 0.001589059829711914 seconds

SyntaxError: invalid syntax (<ipython-input-12-369b74163d9d>, line 1)

In [14]:
locate_grid(xvals, bignodes)

array([ 8181,  8182,  8182, ..., 54543, 54544, 54544])

In [15]:
locate_grid(xvals, bignodes)

array([ 8181,  8182,  8182, ..., 54543, 54544, 54544])

In [8]:
locate_grid(xvals[::-1], bignodes)[::-1]

array([ 81,  82,  82,  83,  83,  84,  84,  84,  85,  85,  86,  86,  87,
        87,  88,  88,  89,  89,  90,  90,  91,  91,  91,  92,  92,  93,
        93,  94,  94,  95,  95,  96,  96,  97,  97,  97,  98,  98,  99,
        99, 100, 100, 101, 101, 102, 102, 103, 103, 103, 104, 104, 105,
       105, 106, 106, 107, 107, 108, 108, 109, 109, 110, 110, 110, 111,
       111, 112, 112, 113, 113, 114, 114, 115, 115, 116, 116, 116, 117,
       117, 118, 118, 119, 119, 120, 120, 121, 121, 122, 122, 122, 123,
       123, 124, 124, 125, 125, 126, 126, 127, 127, 128, 128, 129, 129,
       129, 130, 130, 131, 131, 132, 132, 133, 133, 134, 134, 135, 135,
       135, 136, 136, 137, 137, 138, 138, 139, 139, 140, 140, 141, 141,
       142, 142, 142, 143, 143, 144, 144, 145, 145, 146, 146, 147, 147,
       148, 148, 148, 149, 149, 150, 150, 151, 151, 152, 152, 153, 153,
       154, 154, 154, 155, 155, 156, 156, 157, 157, 158, 158, 159, 159,
       160, 160, 161, 161, 161, 162, 162, 163, 163, 164, 164, 16